In [1]:
import requests
import pandas as pd
import time
import xlwings as xw

In [2]:
def fetch_crypto_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 50,
        "page": 1,
        "sparkline": "false",
        "price_change_percentage": "24h"
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return []


In [3]:
def analyze_data(data):
    df = pd.DataFrame(data)
    df = df[["name", "symbol", "current_price", "market_cap", "total_volume", "price_change_percentage_24h"]]
    df.columns = ["Name", "Symbol", "Current Price (USD)", "Market Cap", "24h Trading Volume", "Price Change (24h %)"]
    
    top_5 = df.nlargest(5, "Market Cap")
    avg_price = df["Current Price (USD)"].mean()
    highest_change = df.loc[df["Price Change (24h %)"].idxmax()]
    lowest_change = df.loc[df["Price Change (24h %)"].idxmin()]
    
    return df, top_5, avg_price, highest_change, lowest_change

In [4]:
def update_excel():
    data = fetch_crypto_data()
    if not data:
        print("No data fetched.")
        return
    
    df, top_5, avg_price, highest_change, lowest_change = analyze_data(data)
    
    wb = xw.Book("crypto_live.xlsx")
    sheet1 = wb.sheets["Live Data"]
    sheet2 = wb.sheets["Analysis"]
    
    sheet1.clear()
    sheet1["A1"].value = df
    
    sheet2.clear()
    sheet2["A1"].value = "Top 5 Cryptocurrencies by Market Cap"
    sheet2["A3"].value = top_5
    sheet2["A10"].value = f"Average Price of Top 50: ${avg_price:.2f}"
    sheet2["A12"].value = "Highest 24h Change"
    sheet2["A13"].value = highest_change
    sheet2["A15"].value = "Lowest 24h Change"
    sheet2["A16"].value = lowest_change
    
    print("Excel updated successfully!")

In [ ]:
if __name__ == "__main__":
    print("Starting live data update...")
    try:
        xw.Book("crypto_live.xlsx")
    except:
        wb = xw.Book()
        wb.sheets.add("Live Data")
        wb.sheets.add("Analysis")
        wb.sheets["Sheet1"].delete()
        wb.save("crypto_live.xlsx")
        wb.close()
    
    while True:
        update_excel()
        time.sleep(300)

Starting live data update...
Excel updated successfully!
